In [1]:
pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

options = Options()
options.add_argument("--headless")
path = r'C:\chromedriver-win64\chromedriver.exe'
service = Service(path)
service.start()

driver = webdriver.Chrome(path, options=options)
driver.get("https://coinmarketcap.com/")

rows = driver.find_elements(By.XPATH, '//table[contains(@class, "cmc-table")]//tbody/tr')

for row in rows[:10]:
    cols = row.find_elements(By.TAG_NAME, 'td')
    rank = cols[1].text
    name = cols[2].text
    price = cols[3].text
    hour_change = cols[4].text
    day_change = cols[5].text
    week_change = cols[6].text
    market_cap = cols[7].text
    volume = cols[8].text

    print(f"{rank} - {name} - {price}")

2 - Ethereum
ETH
Buy - $1,829.06
3 - Tether
USDT
Buy - $1.00
4 - XRP
XRP
Buy - $2.19
5 - BNB
BNB
Buy - $588.72
6 - Solana
SOL
Buy - $146.83
7 - USDC
USDC
Buy - $1.00
8 - Dogecoin
DOGE
Buy - $0.1732
9 - Cardano
ADA
Buy - $0.6893
10 - TRON
TRX
Buy - $0.2476
11 - Sui
SUI
Buy - $3.27
